In [33]:
import time
import math
import numpy as np
import torch
from torch import nn,optim
import torch.nn.functional as F
import sys
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 获取数据

In [34]:
import zipfile
with zipfile.ZipFile('./Datasets/jaychou_lyrics.txt.zip') as zin:
    with zin.open('jaychou_lyrics.txt') as f:
        corpus_chars=f.read().decode('utf-8')
corpus_chars=corpus_chars.replace('\n',' ').replace('\r',' ')
corpus_chars=corpus_chars[0:10000]
idx_to_char=list(set(corpus_chars))
char_to_idx=dict([(char,i) for i,char in enumerate(idx_to_char)])
vocab_size=len(char_to_idx)
corpus_indices=[char_to_idx[char] for char in corpus_chars]

# 定义模型

In [35]:
def one_hot(x,n_class,dtype=torch.float32):
    # X shape: (batch), output shape: (batch, n_class)
    x=x.long()
    res=torch.zeros(x.shape[0],n_class,dtype=dtype,device=x.device)
    res.scatter_(1,x.view(-1,1),1)
    return res
def to_onehot(X,n_class):
    # X shape: (batch, seq_len), output: seq_len elements of (batch,n_class)
    return [one_hot(X[:,i],n_class) for i in range(X.shape[1])]

In [36]:
num_hiddens=256
rnn_layer=nn.RNN(input_size=vocab_size,hidden_size=num_hiddens)

In [37]:
num_steps=35
batch_size=2
state=None
X=torch.rand(num_steps,batch_size,vocab_size)
Y,state_new=rnn_layer(X,state)
print(Y.shape,len(state_new),state_new[0].shape)

torch.Size([35, 2, 256]) 1 torch.Size([2, 256])


In [38]:
class RNNModel(nn.Module):
    def __init__(self,rnn_layer,vocab_size):
        super(RNNModel,self).__init__()
        self.rnn=rnn_layer
        self.hidden_size=rnn_layer.hidden_size*(2 if rnn_layer.bidirectional else 1)
        self.vocab_size=vocab_size
        self.dense=nn.Linear(self.hidden_size,vocab_size)
        self.state=None
    def forward(self,inputs,state):
        #inputs: (batch, seq_len)
        X=to_onehot(inputs,self.vocab_size)
        Y,self.state=self.rnn(torch.stack(X),state)
        output=self.dense(Y.view(-1,Y.shape[-1]))
        return output,self.state
        

# 训练模型

In [39]:
def predict_rnn_pytorch(prefix,num_chars,model,vocab_size,device,idx_to_char,char_to_idx):
    state=None
    output=[char_to_idx[prefix[0]]]
    for t in range(num_chars+len(prefix)-1):
        X=torch.tensor([output[-1]],device=device).view(1,1)
        if state is not None:
            if isinstance(state,tuple):
                state=(state[0].to(device),state[1].to(device))
            else:
                state=state.to(device)
        (Y,state)=model(X,state)
        if t<len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(dim=1).item()))
    return ''.join([idx_to_char[i] for i in output])

In [40]:
model=RNNModel(rnn_layer,vocab_size).to(device)
predict_rnn_pytorch('你好',10,model,vocab_size,device,idx_to_char,char_to_idx)

'你好榜仪榜榜仪术榜榜仪术'

In [41]:
def data_iter_consecutive(corpus_indices,batch_size,num_steps,device=None):
    if device is None:
        device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    corpus_indices=torch.tensor(corpus_indices,dtype=torch.float32,device=device)
    data_len=len(corpus_indices)
    batch_len=data_len//batch_size
    indices=corpus_indices[0:batch_size*batch_len].view(batch_size,batch_len)
    epoch_size=(batch_len-1)//num_steps
    for i in range(epoch_size):
        i=i*num_steps
        X=indices[:,i:i+num_steps]
        Y=indices[:,i+1:i+num_steps+1]
        yield X,Y

In [42]:
def grad_clipping(params,theta,device):
    norm=torch.tensor([0.0],device=device)
    for param in params:
        norm+=(param.grad.data**2).sum()
    norm=norm.sqrt().item()
    if norm>theta:
        for param in params:
            param.grad.data*=(theta/norm)

In [43]:
def train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes):
    loss=nn.CrossEntropyLoss()
    optimizer=torch.optim.Adam(model.parameters(),lr=lr)
    model.to(device)
    state=None
    for epoch in range(num_epochs):
        l_sum,n,start=0.0,0,time.time()
        data_iter=data_iter_consecutive(corpus_indices,batch_size,num_steps,device)
        for X,Y in data_iter:
            if state is not None:
                if isinstance(state,tuple):# LSTM, state:(h, c)
                    state=(state[0].detach(),state[1].detach())
                else:
                    state=state.detach()
            (output,state)=model(X,state)
            y=torch.transpose(Y,0,1).contiguous().view(-1)
            l=loss(output,y.long())
            optimizer.zero_grad()
            l.backward()
            grad_clipping(model.parameters(),clipping_theta,device)
            optimizer.step()
            l_sum+=l.item()*y.shape[0]
            n+=y.shape[0]
        try:
            perplexity=math.exp(l_sum/n)
        except OverflowError:
            perplexity=float('inf')
        if (epoch+1)%pred_period==0:
            print('epoch %d,perplexity %f,time %.2f sec '%(epoch+1,perplexity,time.time()-start))
            for prefixe in prefixes:
                print(' -',predict_rnn_pytorch(prefixe,pred_len,model,vocab_size,device,idx_to_char,char_to_idx))

In [44]:
num_epochs,batch_size,lr,clipping_theta=250,32,1e-3,1e-2
pred_period,pred_len,prefixes=50,50,['你好','不好']

In [45]:
train_and_predict_rnn_pytorch(model,num_hiddens,vocab_size,device,corpus_indices,idx_to_char,char_to_idx,num_epochs,num_steps,lr,clipping_theta,batch_size,pred_period,pred_len,prefixes)

epoch 50,perplexity 9.966159,time 0.18 sec 
 - 你好多   你不多 离 我不能再想 我不要再想 我想 我不能再想 我不能再想 我不能再想 我不能再想 我
 - 不好 我 我有多烦恼  想要你这样打我  不要再这样牵着你的可爱女人 坏坏的让我疯狂的可爱女人 坏坏的让
epoch 100,perplexity 1.296433,time 0.17 sec 
 - 你好多  你的回忆 找不着 不  不要再这样打我妈妈 我说你爸 你是我妈妈你 种爸你 爸我妈手 不要再 
 - 不好 你 我不了这节奏 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离
epoch 150,perplexity 1.067609,time 0.18 sec 
 - 你好多球 想 你说的黑我笑能想要你  不是 不想开你看着我不起 说有话双截棍 哼哼哈兮 习武之人切记 仁
 - 不好 你 我不了口不能痛吗 就是你 家你一样看着日的 一直到我 你这样我我想知不觉 你已经离开我 不知不
epoch 200,perplexity 1.052536,time 0.17 sec 
 - 你好多你 想 你说的爱 有多难熬  没有你在我有多难熬多烦恼  没有你烦 我有多烦恼  没有你烦我有多烦
 - 不好 你 我不了口不能痛吗 我不了 爱你在这样对简活 我想你 说你怎么面对我 甩开球我满腔的怒火 我想揍
epoch 250,perplexity 1.021723,time 0.18 sec 
 - 你好多 我想你 说你开始 打我妈 你说我 有不再痛  一切中轻轻  色作  后是一只都有轻的叹息  后悔
 - 不好 你 我不了口不作痛  我不要 你你 我不会痛不 我不多难熬  我将可以 从 有话去对医药箱说 别怪
